## Random Polygon Generation

In [ ]:
from cyaron import Polygon, Vector
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw


### Visualization

In [ ]:
def draw_polygon(poly_point_list):
    jet = plt.cm.jet
    colors = jet(np.linspace(0, 1, len(poly_point_list)))

    # plot only polygon, no axis or frame
    fig = plt.figure(frameon=False)
    ax = plt.gca()
    ax.axis('off')
    ax.axis('equal')
    outer_vxs = poly_point_list
    wall = np.vstack((outer_vxs, outer_vxs[0]))
    wall = np.vstack((wall, np.array([np.nan, np.nan])))

    plt.plot(wall[:, 0], wall[:, 1], 'black')
    def plot_vertices(vertices):
        for index, vx in enumerate(vertices):
            plt.scatter(vx[0], vx[1], color=colors[index], s=60)

    plt.show()

## Method 1: Use the Cyaron library

Generate a random set of points. Find a random poly line connecting the points by divide and conquer. The documentation for the library is not very good and only in Chinese, but the source [here](https://github.com/luogu-dev/cyaron/blob/master/cyaron/polygon.py) is not so hard to read.

In [ ]:
# Vector.random(num=5, position_range=[10], mode=0, **kwargs):
# mode 0=unique 1=repeatable 2=float
poly = Polygon.simple_polygon(Vector.random(20, [400, 400], 2)).points
draw_polygon(poly)

## Method 2: the Stackoverflow answer with spikyness control. [Source](https://stackoverflow.com/questions/8997099/algorithm-to-generate-random-2d-polygon)

This method can only generate star shaped polygons. 

In [ ]:
import math, random

def generatePolygon( ctrX, ctrY, aveRadius, irregularity, spikeyness, numVerts ) :
    '''Start with the centre of the polygon at ctrX, ctrY, 
    then creates the polygon by sampling points on a circle around the centre. 
    Randon noise is added by varying the angular spacing between sequential points,
    and by varying the radial distance of each point from the centre.

    Params:
    ctrX, ctrY - coordinates of the "centre" of the polygon
    aveRadius - in px, the average radius of this polygon, this roughly controls how large the polygon is, really only useful for order of magnitude.
    irregularity - [0,1] indicating how much variance there is in the angular spacing of vertices. [0,1] will map to [0, 2pi/numberOfVerts]
    spikeyness - [0,1] indicating how much variance there is in each vertex from the circle of radius aveRadius. [0,1] will map to [0, aveRadius]
    numVerts - self-explanatory

    Returns a list of vertices, in CCW order.
    '''

    a = clip( irregularity, 0,1 ) * 2*math.pi / numVerts
    spikeyness = clip( spikeyness, 0,1 ) * aveRadius

    # generate n angle steps
    angleSteps = []
    lower = (2*math.pi / numVerts) - irregularity
    upper = (2*math.pi / numVerts) + irregularity
    sum = 0
    for i in range(numVerts) :
        tmp = random.uniform(lower, upper)
        angleSteps.append( tmp )
        sum = sum + tmp

    # normalize the steps so that point 0 and point n+1 are the same
    k = sum / (2*math.pi)
    for i in range(numVerts) :
        angleSteps[i] = angleSteps[i] / k

    # now generate the points
    points = []
    angle = random.uniform(0, 2*math.pi)
    for i in range(numVerts) :
        r_i = clip( random.gauss(aveRadius, spikeyness), 0, 2*aveRadius )
        x = ctrX + r_i*math.cos(angle)
        y = ctrY + r_i*math.sin(angle)
        points.append( (int(x),int(y)) )

        angle = angle + angleSteps[i]

    return points

def clip(x, min, max) :
    if( min > max ) :  return x    
    elif( x < min ) :  return min
    elif( x > max ) :  return max
    else :             return x

In [ ]:
verts = generatePolygon( ctrX=250, ctrY=250, aveRadius=100, irregularity=0.35, spikeyness=0.5, numVerts=16 )
draw_polygon(verts)

## Method 3: RGP - Generation of Random Polygonal Objects

The paper is [here]( http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.56.5609&rep=rep1&type=pdf). For some reason the webpage for this project is not accessible anymore, so here is the Internet Wayback [link](https://web.archive.org/web/20190918003901/http://www.cosy.sbg.ac.at/~held/projects/rpg/rpg.html). I haven't read the paper yet but from the webpage their results is much more systematic than the above too. So it might worth to ask them for the code. 